In [1]:
# Install Hugging Face Transformers for models and tokenizers
# Install Hugging Face Datasets for easy data loading
# Install accelerate for faster training with Hugging Face Trainer
# Install scikit-learn for evaluation metrics (like accuracy)

!pip install transformers datasets accelerate scikit-learn

In [2]:
from datasets import load_dataset

# Downgrade fsspec to resolve the glob pattern issue
# This is a temporary fix for a known compatibility issue
!pip install fsspec==2023.9.0

# Load the full IMDB dataset
# This might take a minute or two depending on your internet connection
print("Loading dataset...")
full_dataset = load_dataset("imdb")
print("Dataset loaded!")

# --- IMPORTANT FOR SPEED ---
# To make this project finish quickly, we'll use a small subset of the data.
# This is crucial for completing it in less than a day.
# In a real project, you'd use all data, but for learning, this is perfect.

# Select the first 1000 samples for training and 200 for testing
train_dataset = full_dataset["train"].shuffle(seed=42).select(range(1000))
eval_dataset = full_dataset["test"].shuffle(seed=42).select(range(200)) # Using 'test' for evaluation

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

# Let's look at one example
print("\nExample from training data:")
print(train_dataset[0])

Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset loaded!
Training samples: 1000
Evaluation samples: 200

Example from training data:
{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer for DistilBERT
# This converts text into numbers that the model understands
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer loaded!")

# Load the DistilBERT model pre-trained for sequence classification
# num_labels=2 because we have two categories: positive (1) and negative (0)
print("Loading model...")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
print("Model loaded!")

Loading tokenizer...
Tokenizer loaded!
Loading model...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded!


In [4]:
def tokenize_function(examples):
    # Apply the tokenizer to the 'text' column of the dataset
    # truncation=True: Cut off reviews longer than the model's max input length (usually 512 tokens)
    # padding=True: Pad shorter reviews with zeros so all inputs have the same length (important for batching)
    return tokenizer(examples["text"], truncation=True, padding=True)

print("Tokenizing datasets...")
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
print("Tokenization complete!")

# --- Prepare data for the Hugging Face Trainer ---
# The Trainer expects input columns named 'input_ids', 'attention_mask', and 'labels'.
# Our dataset has 'text' and 'label', so we need to rename/remove columns.
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(["text"])

# Rename 'label' column to 'labels'
tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
tokenized_eval_dataset = tokenized_eval_dataset.rename_column("label", "labels")

# Set the format to PyTorch tensors
tokenized_train_dataset.set_format("torch")
tokenized_eval_dataset.set_format("torch")

print("\nExample of tokenized data (first 50 input_ids):")
print(tokenized_train_dataset[0]['input_ids'][:50])
print(tokenized_train_dataset[0]['labels'])

Tokenizing datasets...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenization complete!

Example of tokenized data (first 50 input_ids):
tensor([  101,  2045,  2003,  2053,  7189,  2012,  2035,  2090,  3481,  3771,
         1998,  6337,  2099,  2021,  1996,  2755,  2008,  2119,  2024,  2610,
         2186,  2055,  6355,  6997,  1012,  6337,  2099,  3504, 15594,  2100,
         1010,  3481,  3771,  3504,  4438,  1012,  6337,  2099, 14811,  2024,
         3243,  3722,  1012,  3481,  3771,  1005,  1055,  5436,  2024,  2521])
tensor(1)


In [5]:
!pip install --upgrade datasets huggingface_hub

In [13]:
import numpy as np
# Import load_metric from the evaluate library instead of datasets
from evaluate import load

# Load the accuracy metric
accuracy_metric = load("accuracy")

# Function to compute metrics for the Trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert model's output (logits) to predicted class IDs (0 or 1)
    predictions = np.argmax(logits, axis=-1)
    # Compute accuracy using the loaded metric
    return accuracy_metric.compute(predictions=predictions, references=labels)

print("Metrics function defined.")

Metrics function defined.


In [15]:
from transformers import TrainingArguments, Trainer

# Define training arguments
# These control various aspects of the training process
training_args = TrainingArguments(
    output_dir="./results", # Directory to save checkpoints and results
    eval_strategy="epoch", # Evaluate performance after each epoch
    learning_rate=2e-5, # Controls how big of a step the model takes when updating its weights
    per_device_train_batch_size=16, # Number of samples processed in one training step
    per_device_eval_batch_size=16, # Number of samples processed in one evaluation step
    num_train_epochs=3, # Number of times to loop through the entire training dataset (KEEP THIS LOW for speed!)
    weight_decay=0.01, # Regularization to prevent overfitting
    logging_dir='./logs', # Directory for logs
    logging_steps=10, # Log progress every 10 steps
    report_to="none" # Disable reporting to external tools for simplicity
)

# Create the Trainer instance
# This object orchestrates the entire training process
trainer = Trainer(
    model=model, # The model we want to fine-tune
    args=training_args, # The training arguments we defined
    train_dataset=tokenized_train_dataset, # Our prepared training data
    eval_dataset=tokenized_eval_dataset, # Our prepared evaluation data
    tokenizer=tokenizer, # The tokenizer (used for padding if not done already)
    compute_metrics=compute_metrics, # Our function to calculate metrics
)

# Start training! This will take some time (minutes, not hours, with subsetted data)
print("Starting training...")
trainer.train()
print("Training complete!")

# Evaluate the final model on the evaluation set
print("\nEvaluating final model...")
results = trainer.evaluate()
print(results)

/tmp/ipython-input-15-2581407738.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.489900,0.448035,0.810000
2,0.289300,0.328636,0.860000
3,0.237100,0.345078,0.860000


Training complete!

Evaluating final model...


{'eval_loss': 0.3450782895088196, 'eval_accuracy': 0.86, 'eval_runtime': 176.3199, 'eval_samples_per_second': 1.134, 'eval_steps_per_second': 0.074, 'epoch': 3.0}


In [16]:
from transformers import pipeline

# Create a sentiment analysis pipeline using your fine-tuned model and tokenizer
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0) # device=0 means use GPU if available

# Test with some new reviews
reviews = [
    "This movie was absolutely fantastic! I loved every moment of it. Highly recommend.",
    "Boring and predictable. I fell asleep halfway through. A complete waste of time.",
    "It was okay, not great, not terrible. Just average.",
    "The acting was superb, but the plot was a bit confusing."
]

print("\n--- Making predictions on new text ---")
for review in reviews:
    result = sentiment_analyzer(review)[0] # Get the first (and only) result
    print(f"Review: '{review}'")
    print(f"Prediction: {result['label']} (Score: {result['score']:.4f})\n")

Device set to use cpu



--- Making predictions on new text ---
Review: 'This movie was absolutely fantastic! I loved every moment of it. Highly recommend.'
Prediction: LABEL_1 (Score: 0.9284)

Review: 'Boring and predictable. I fell asleep halfway through. A complete waste of time.'
Prediction: LABEL_0 (Score: 0.9525)

Review: 'It was okay, not great, not terrible. Just average.'
Prediction: LABEL_0 (Score: 0.8699)

Review: 'The acting was superb, but the plot was a bit confusing.'
Prediction: LABEL_0 (Score: 0.5086)



In [17]:
# Save the model and tokenizer to a directory
output_model_dir = "./my_fine_tuned_sentiment_model"
trainer.save_model(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

print(f"Model and tokenizer saved to: {output_model_dir}")

Model and tokenizer saved to: ./my_fine_tuned_sentiment_model
